# Project 1 - Room Booking App #
### Dhrov Subramanian & Yeggi Lee ###

## Creating the tables ##

1. users
2. venues
3. events
4. confirmedEvents


In [2]:
import sqlite3
import pandas as pd

#db=sqlite3.connect("/Users/ylee/Desktop/sqlite/test.db")
db=sqlite3.connect("/sqlite3/test.db")
cursor = db.cursor()

cursor.execute('''
    DROP TABLE IF EXISTS users''');
cursor.execute('''
    DROP TABLE IF EXISTS venues''');
cursor.execute('''
    DROP TABLE IF EXISTS events''');
cursor.execute('''
    DROP TABLE IF EXISTS confirmedEvents''');
cursor.execute('''
    DROP TABLE IF EXISTS Time''');

#Make email unique after testing please. 

cursor.execute('''
    CREATE TABLE users(
                       user_id INTEGER PRIMARY KEY, 
                       name TEXT,
                       phone TEXT, 
                       email TEXT UNIQUE, 
                       EID TEXT UNIQUE,
                       admin BOOLEAN)
''');

cursor.execute('''
    CREATE TABLE venues(
                        venue_id INTEGER PRIMARY KEY, 
                        bldg_code TEXT,
                        floor_num INTEGER,
                        room_num INTEGER, 
                        room_capacity INTEGER,
                        open_time TEXT,
                        close_time TEXT,
                        time_id INTEGER,
                        CONSTRAINT 
                             fk_events FOREIGN KEY (time_id) 
                             REFERENCES time(time_id))
''');

cursor.execute('''
    CREATE TABLE events(
                        event_id INTEGER PRIMARY KEY, 
                        name TEXT, 
                        description TEXT,  
                        capacity INTEGER, 
                        venue_id INTEGER,
                        event_owner INTEGER,
                        time_id INTEGER,
                        CONSTRAINT 
                             fk_events FOREIGN KEY (time_id) 
                             REFERENCES time(time_id),
                        CONSTRAINT 
                            fk_venues FOREIGN KEY (venue_id) 
                            REFERENCES venues(venue_id),
                        CONSTRAINT 
                            fk_users FOREIGN KEY (event_owner) 
                            REFERENCES users(user_id))
                   
''');

cursor.execute('''
    CREATE TABLE confirmedEvents(
                                 confirmedEvents_id INTEGER PRIMARY KEY, 
                                 event_id INTEGER, user_id INTEGER,
                                 CONSTRAINT 
                                     fk_events FOREIGN KEY (event_id) 
                                     REFERENCES events(event_id),
                                 CONSTRAINT 
                                     fk_users FOREIGN KEY (user_id) 
                                     REFERENCES users(user_id))
''');

cursor.execute('''
    CREATE TABLE Time(
                       time_id INTEGER PRIMARY KEY, 
                       event_id INTEGER, venue_id INTEGER,
                         CONSTRAINT 
                             fk_events FOREIGN KEY (event_id) 
                             REFERENCES events(event_id),
                         CONSTRAINT 
                             fk_users FOREIGN KEY (venue_id) 
                             REFERENCES venue(venue_id))
''');

db.commit()

In the section below, we're adding test subjects to check that the tables are working correctly.

In [3]:
import datetime
dob1 = datetime.date(1991, 1,1)

cursor.execute('''INSERT INTO users(name, phone, email, EID, admin)
                  VALUES(:name,:phone, :email, :EID, :admin)''',
                  {'name':'Joe', 'phone':'222-333-4444', 'email':'joe2@email.com', 'EID': 'ds39547','admin': 0 })

cursor.execute('''INSERT INTO users(name, phone, email, EID, admin)
                  VALUES(:name,:phone, :email, :EID, :admin)''',
                  {'name':'Jamie', 'phone':'111-111-111', 'email':'jamielee@email.com', 'EID': 'ds39548' ,'admin': 1})

cursor.execute('''INSERT INTO venues(bldg_code, floor_num, room_num, room_capacity, open_time, close_time)
                  VALUES(:bldg_code, :floor_num, :room_num, :room_capacity, :open_time, :close_time)''',
                  {'bldg_code':'PCL', 'floor_num':'3','room_num':'104', 'room_capacity':'30', 'open_time':'07:00', 'close_time':'21:00'})



In [4]:
#To print out all elements in the users table
for row in cursor.execute('''SELECT * FROM users'''):
    print(row)
    
for row in cursor.execute('''SELECT * FROM venues'''):
    print(row)
    
for row in cursor.execute('''SELECT * FROM events'''):
    print(row)

(1, 'Joe', '222-333-4444', 'joe2@email.com', 'ds39547', 0)
(2, 'Jamie', '111-111-111', 'jamielee@email.com', 'ds39548', 1)
(1, 'PCL', 3, 104, 30, '07:00', '21:00', None)


## Adding a user ## 

### Parameters: ###
1. **db**: database
2. **name**: name of the user
3. **phone**: phone of the user
4. **email**: email of the user (must be unique)
5. **EID**: EID of the user
6. **admin**: checks whether the user is an admin or not (0 = False, 1 = True)

### Sequence: ###
1. Using the user_id, the function fetches the information of that entry 
2. With that info, the function is able to determine if the user is an admin or not
3. If the user is an admin, the user is added to the database
4. If the user is not an admin, an error is raised 

### Add a new user (admin only)

In [5]:
def adduser(user_id, name, phone, email, EID, admin):
    ## Grabs the row of information that connects to the user_id
    adminCheck = cursor.execute('''SELECT * FROM users WHERE user_id = ? ''', (user_id,))
    entry = cursor.fetchone()

    ## Checking that the user is an admin
    if entry[5] == 1: 
        sql = "INSERT INTO users(name, phone, email, EID, admin) VALUES(?, ?, ?, ?, ?)"
        cursor.execute(sql, (name, phone, email, EID, admin))
        db.commit()
        return('New user added')
    else:
        raise Exception('ERROR: Not an admin') 

    
adduser(2, 'Bob', '222-222-222', 'bobthebuilder@gmail.com', '1997-01-11', 0)


print(pd.read_sql_query("SELECT * FROM users", db))


   user_id   name         phone                    email         EID  admin
0        1    Joe  222-333-4444           joe2@email.com     ds39547      0
1        2  Jamie   111-111-111       jamielee@email.com     ds39548      1
2        3    Bob   222-222-222  bobthebuilder@gmail.com  1997-01-11      0


## Adding a venue ## 

### Parameters: ###
1. **user_id**: id of the user (primary key)
2. **bldg_code**: building code of the building (should be 3 letters)
3. **floor_num**: floor of the room
4. **room_num**: number of the room
5. **room_capacity**: how many people can fit in the room

### Sequence: ###
1. Using the user_id, the function fetches the information of that entry 
2. With that info, the function is able to determine if the user is an admin or not
3. If the user is an admin, the venue is added to the database
4. If the user is not an admin, an error is raised 

In [6]:
## Adding a venue
## TODO: Find a way to make this admin only

def addvenue(user_id, bldg_code, floor_num, room_num, room_capacity):
    ## Grabs the row of information that connects to the user_id
    adminCheck = cursor.execute('''SELECT * FROM users WHERE user_id = ? ''', (user_id,))
    entry = cursor.fetchone()

    ## Checking that the user is an admin
    if entry[5] == 1: 
        sql = "INSERT INTO venues(bldg_code, floor_num, room_num, room_capacity) VALUES(?, ?, ?, ?)"
        cursor.execute(sql, (bldg_code, floor_num, room_num, room_capacity))
        db.commit()
        return('New venue added')
    else:
        raise Exception('ERROR: Not an admin') 

addvenue(2, 'CBA', 4, 211, 40)


print(pd.read_sql_query("SELECT * FROM venues", db))

   venue_id bldg_code  floor_num  room_num  room_capacity open_time  \
0         1       PCL          3       104             30     07:00   
1         2       CBA          4       211             40      None   

  close_time time_id  
0      21:00    None  
1       None    None  


## Start an Event ## 

### Parameters: ###

### Sequence: ###

In [7]:
## start an event
from datetime import datetime, timedelta
'''
    sql_Vcapacity = "Select room_capacity FROM venues WHERE venues.venue_id = event.venue_id"
    #^^ this line sux
    if capacity > int(sql_Vcapacity):
        sql = "INSERT INTO events(name, description, start_time, event_time, capacity, venue_id, event_owner) VALUES(?, ?, ?, ?, ?, ?, ?)"
        cursor = db.cursor()
        cursor.execute(sql, (name, description, start_time, event_time, capacity, venue_id, event_owner))
        return('New event added')
    else:
        raise Exception('ERROR: Room capacity exceeded')    
'''

def addevent(db, name, description, start_time, event_time, capacity, venue_id, event_owner):
    sql = "INSERT INTO events(name, description, start_time, event_time, capacity, venue_id, event_owner) VALUES(?, ?, ?, ?, ?, ?, ?)"
    cursor = db.cursor()
    cursor.execute(sql, (name, description, start_time, event_time, capacity, venue_id, event_owner))
    return "boot"

addevent(db, 'test', 'a test of the system', datetime(2016, 9, 1, 8), 2,  30, 1, 2)

print(pd.read_sql_query("SELECT * FROM events", db))

   event_id  name           description           start_time  event_time  \
0         1  test  a test of the system  2016-09-01 08:00:00           2   

   capacity  venue_id  event_owner time_id  
0        30         1            2    None  


In [8]:
print(pd.read_sql_query("SELECT * FROM confirmedEvents", db))

Empty DataFrame
Columns: [confirmedEvents_id, event_id, user_id]
Index: []


### Display timeslot availability at a venue 

In [9]:
#Trying to figure out timeslots
from datetime import datetime, timedelta
import pandas as pd

def datetime_range(start, end, delta):
    while start < end:
        yield start
        start += delta

sql_time = "Select start_time FROM events"
sql_time2 = "Select event_time FROM events"

for row in cursor.execute(sql_time):
    dts =[]
    start = (pd.to_datetime(row));
    dts += [dt.strftime('%Y-%m-%d %H:%M') for dt in 
           datetime_range(start , datetime(2016, 9, 1, 9+12), 
           timedelta(minutes=60))]
    print(dts)
    


[Index(['2016-09-01 08:00'], dtype='object'), Index(['2016-09-01 09:00'], dtype='object'), Index(['2016-09-01 10:00'], dtype='object'), Index(['2016-09-01 11:00'], dtype='object'), Index(['2016-09-01 12:00'], dtype='object'), Index(['2016-09-01 13:00'], dtype='object'), Index(['2016-09-01 14:00'], dtype='object'), Index(['2016-09-01 15:00'], dtype='object'), Index(['2016-09-01 16:00'], dtype='object'), Index(['2016-09-01 17:00'], dtype='object'), Index(['2016-09-01 18:00'], dtype='object'), Index(['2016-09-01 19:00'], dtype='object'), Index(['2016-09-01 20:00'], dtype='object')]


In [10]:
sql_time = "Select open_time FROM venues"
sql_time2 = "Select close_time FROM venues" 

for row in cursor.execute(sql_time):
    start = datetime.strptime(row, "%H:%M");
    print (start)
    



TypeError: strptime() argument 1 must be str, not tuple